In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)

In [5]:

mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

lambda_vector = np.array([0, 0.0, 0.015])

cavity_options = {
    'omega_value' : 0.12,
    'lambda_vector' : np.array([0, 0, 0.015]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    # specify the number of photons - 2 means |0> , |1>, |2> will be in the basis 
    'number_of_photons' : 0
}


# Build normal CI matrix
LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_options)

# update number of photons
cavity_options['number_of_photons'] = 3

# Build PF-CI matrix with 'number_of_photons' : 3
LiH_Np_3 = PFHamiltonianGenerator(mol_str, options_dict, cavity_options)


Start SCF iterations:

Canonical RHF One-electron energy = -12.2195250859902877
CQED-RHF One-electron energy      = -12.2179005272759067
Nuclear repulsion energy          = 0.8819620177833333
Dipole energy                     = 0.0014814292149263
SCF Iteration   1: Energy = -7.8495723586378929   dE = -7.84957E+00   dRMS = 3.58632E-05
SCF Iteration   2: Energy = -7.8495725043087159   dE = -1.45671E-07   dRMS = 1.36143E-05
SCF Iteration   3: Energy = -7.8495725326124912   dE = -2.83038E-08   dRMS = 7.27429E-06
SCF Iteration   4: Energy = -7.8495725404413470   dE = -7.82886E-09   dRMS = 3.85006E-06
SCF Iteration   5: Energy = -7.8495725426206029   dE = -2.17926E-09   dRMS = 2.03271E-06
SCF Iteration   6: Energy = -7.8495725432273744   dE = -6.06772E-10   dRMS = 1.07274E-06
SCF Iteration   7: Energy = -7.8495725433963273   dE = -1.68953E-10   dRMS = 5.66085E-07
SCF Iteration   8: Energy = -7.8495725434433670   dE = -4.70397E-11   dRMS = 2.98720E-07
SCF Iteration   9: Energy = -7.849572543

Print the shapes of each matrix... LiH should be ordinary FCI matrix and 
LiH_Np_1 should be the PF-FCI in the basis of $|0\rangle$, $|1\rangle$, $|2\rangle$, $|3\rangle$ photon states

In [6]:
print(np.shape(LiH.H_PF))
print(np.shape(LiH_Np_3.H_PF))

(225, 225)
(900, 900)


To-Do 1: Inspect the structure of the Hamiltonian blocks.  The "diagonal" blocks with the same photon-number state $|n\rangle$ on the bra and the ket should be the same as the ordinary FCI matrix except for an added factor of $n \cdot \omega{\bf 1}$.  The "off-diagonal" bocks from $\langle n | \hat{H} | n+1 \rangle$ will have the form 
$\sqrt{n+1} {\bf G} + \sqrt{n+1}\langle G \rangle {\bf 1}$ and $\langle n+1 | \hat{H} | n \rangle$ will have the form 
$\sqrt{n} {\bf G} + \sqrt{n}\langle G \rangle {\bf 1}$

In [ ]:
# check the structure of the blocks here

To-Do 2: Assuming the structure checks out, investigate the behaviour of the LiH stretch as a function of
different number of photon states (trying values of 'number_of_photons' ranging from 0 to 6 if possible)  as compared to the pCQED results pasted below that use 20 electronic states and 20 photonic states, where the 20 electronic states were computed at the sto-3G/FCI level.  The bondlength for each calculation are in row 0 of the array called `pCQED_results`, and the first 4 electronic states are in rows 1-4.

It would be good to provide the PES data for plotting for each 'number_of_photons' value, and also a value for the mean squared error for each PES for each number_of_photons' value.  A template for the bond-length scan and omega/lambda parameters is in the cell below the pCQED results.

In [7]:
# pCQED results stored in this array for comparison to QED-FCI results
pCQED_results = np.array([[1.4, -7.877226809519587, -7.757917465066483,  -7.7343307256227245, -7.7343307256227245],
[1.4163265306122448, -7.878182902550236, -7.7589144666345105,  -7.735842282297653, -7.735842282297653],
[1.4326530612244897, -7.8790030479591575, -7.759778901828479,  -7.737249882024182, -7.737249882024182],
[1.4489795918367345, -7.879694018639106, -7.760517935675106,  -7.738558123439453, -7.738558123439453],
[1.4653061224489796, -7.880262264116102, -7.761138469652131,  -7.7397712706532715, -7.7397712706532715],
[1.4816326530612245, -7.880713929816529, -7.761647171695738,  -7.740893261088976, -7.740893261088976],
[1.4979591836734694, -7.881054874894629, -7.76205050691835,  -7.741927709698678, -7.741927709698678],
[1.5142857142857142, -7.881290688741532, -7.762354769579052,  -7.74287790924611, -7.74287790924611],
[1.530612244897959, -7.8814267062880745, -7.76256611689351,  -7.743746826289557, -7.743746826289557],
[1.546938775510204, -7.881468022205304, -7.762690605296552,  -7.74453709245481, -7.74453709245481],
[1.5632653061224488, -7.8814195040981065, -7.762734229752586,  -7.745250990591799, -7.745250990591799],
[1.5795918367346937, -7.881285804779993, -7.7627029666038965,  -7.745890435498386, -7.745890435498386],
[1.5959183673469388, -7.881071373708463, -7.762602820182364,  -7.746456949145305, -7.746456949145305],
[1.6122448979591837, -7.88078046765399, -7.762439872877222,  -7.746951630855732, -7.746951630855732],
[1.6285714285714286, -7.880417160668269, -7.76222033738602,  -7.74737512384615, -7.74737512384615],
[1.6448979591836734, -7.879985353411742, -7.761950608264389,  -7.747727581135525, -7.747727581135525],
[1.6612244897959183, -7.87948878189391, -7.761637307389326,  -7.748008636318737, -7.748008636318737],
[1.6775510204081634, -7.87893102567524, -7.76128731439683,  -7.748217388245947, -7.748217388245947],
[1.693877551020408, -7.878315515574586, -7.760907768677104,  -7.748352413117, -7.748352413117],
[1.7102040816326531, -7.877645540921302, -7.760506024946395,  -7.748411822056239, -7.748411822056239],
[1.726530612244898, -7.876924256387764, -7.760089541822191,  -7.7483933848180975, -7.7483933848180975],
[1.7428571428571429, -7.8761546884343545, -7.7596656857697806,  -7.748294737326155, -7.748294737326155],
[1.7591836734693878, -7.875339741395601, -7.7592414455490495,  -7.748113677980537, -7.748113677980537],
[1.7755102040816326, -7.874482203233443, -7.758823077087811,  -7.747848532866205, -7.747848532866205],
[1.7918367346938777, -7.8735847509813, -7.758415731171892,  -7.74749853752623, -7.74749853752623],
[1.8081632653061224, -7.872649955899655, -7.758023142752362,  -7.747064156547735, -7.747064156547735],
[1.8244897959183675, -7.871680288362554, -7.75764746266141,  -7.746547260215138, -7.746547260215138],
[1.8408163265306123, -7.870678122492145, -7.757289280393505,  -7.745951109615156, -7.745951109615156],
[1.8571428571428572, -7.869645740556419, -7.756947831092051,  -7.7452801570905105, -7.7452801570905105],
[1.873469387755102, -7.868585337144743, -7.756621328174914,  -7.744539720644809, -7.744539720644809],
[1.889795918367347, -7.8674990231331785, -7.756307339379942,  -7.743735614548488, -7.743735614548488],
[1.906122448979592, -7.866388829451298, -7.756003133295019,  -7.742873809110713, -7.742873809110713],
[1.9224489795918367, -7.8652567106604065, -7.755705952373911,  -7.741960163642176, -7.741960163642176],
[1.9387755102040818, -7.864104548352614, -7.755413199187191,  -7.741000245884286, -7.741000245884286],
[1.9551020408163267, -7.862934154378391, -7.75512254409347,  -7.739999229740413, -7.739999229740413],
[1.9714285714285715, -7.861747273910449, -7.75483197241642,  -7.738961853244534, -7.738961853244534],
[1.9877551020408164, -7.860545588349976, -7.754539790575334,  -7.737892417336881, -7.737892417336881],
[2.0040816326530613, -7.859330718081179, -7.754244607556131,  -7.736794809076253, -7.736794809076253],
[2.0204081632653064, -7.858104225079025, -7.753945303709014,  -7.735672537317424, -7.735672537317424],
[2.036734693877551, -7.856867615375043, -7.7536409947770775,  -7.734528772963997, -7.734528772963997],
[2.053061224489796, -7.855622341384742, -7.753330995903563,  -7.7333663890612785, -7.7333663890612785],
[2.069387755102041, -7.854369804100433, -7.753014788180499,  -7.732187998179384, -7.732187998179384],
[2.085714285714286, -7.8531113551526035, -7.752691988904905,  -7.730995985932141, -7.730995985932141],
[2.1020408163265305, -7.851848298742407, -7.752362325878152,  -7.729792540306601, -7.729792540306601],
[2.1183673469387756, -7.850581893448013, -7.752025615628896,  -7.72857967693312, -7.72857967693312],
[2.1346938775510207, -7.849313353906683, -7.751681745215316,  -7.727359260647067, -7.727359260647067],
[2.1510204081632653, -7.848043852375053, -7.751330657176805,  -7.726133023778703, -7.726133023778703],
[2.1673469387755104, -7.84677452016878, -7.750972337192588,  -7.724902581615083, -7.724902581615083],
[2.183673469387755, -7.845506448984159, -7.750606804032982,  -7.723669445448475, -7.723669445448475],
[2.2, -7.844240692102628, -7.750234101429436,  -7.722435033573057, -7.722435033573057]])

In [12]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

cavity_options = {
    'omega_value' : 0.12,
    'lambda_vector' : np.array([0, 0, 0.025]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    # specify the number of photons - 2 means |0> , |1>, |2> will be in the basis 
    'number_of_photons' : 0
}


# number of bondlengths
N_R = 50
# number of electronic states to store - keeping 20 just in case there are a lot of triplets!
N_el = 20
# bondlengths between 1.4 and 2.2 angstroms
r_array = np.linspace(1.4, 2.2, N_R)

# array for the polariton energies
E_array = np.zeros((N_R, N_el))


r_idx = 0
for r in r_array:
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    # run PF-HF calculation here and analyze the data!

To-Do 2: The cavity parameters were chosen to create strong polaritonic character at the LiH bondlength of 
1.8 angstroms.  Let's investigate the behavior of the ground-state, lower-polariton, and upper-polariton states 
at r=1.8 Angstroms as a function of the value of $\lambda_z$ in the range (0, 0.1) in increments of 0.01
also for different numbers of photon states ranging from 0 to 6.  I don't yet have the corresponding pCQED results, but will work on them.
